In [ ]:
import numpy as np
import sympy as sp
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from scipy.integrate import solve_ivp
from functools import partial

import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [23]:
from tqdm import trange

for _i in trange(10):
    _i ** _i

100%|███████████████████████████████████████| 10/10 [00:00<00:00, 118818.81it/s]


In [ ]:
n3_sim_params = {
    'l1' : 0.34,
    'l2' : 0.33,
    'l3' : 0.33,
    'm1' : 0.33,
    'm2' : 0.33,
    'm3' : 0.34,
    'g' : 9.81,
}

sim_params = {
    'l1' : 0.34,
    'l2' : 0.66,
    'm1' : 0.66,
    'm2' : 0.34,
    'g' : 9.81,
}

ani_params = {
    'tf' : 15,
    'fps' : 120,
    'g' : 9.81,
}

rand_params = {
    'min_q' : 0.0,
    'min_p' : 0.0,
    'max_q' : np.pi / 2,
    'max_p' : np.pi / 6,
}

default_ics = [
    [np.pi / 2, 0.0],
    [np.pi / 2, 0.0, np.pi / 2, 0.0],
    [np.pi / 2, 0.0, np.pi / 2, 0.0, np.pi / 2, 0.0]
]

In [ ]:
rng = np.random.default_rng()

Euler-Lagrange Equations: $$\frac{\partial L}{\partial q_i} - \frac{d}{dt}\left(\frac{\partial L}{\partial \dot{q}_i}\right) = 0$$

In [ ]:
def EulerLagrange(L, q, p):
    try:
        return [
            L.diff(_q) - L.diff(_p).diff(t) for _q,_p in zip(q, p)
        ]
    except:
        return L.diff(q) - L.diff(p).diff(t)

In [ ]:
def pypendula(params, ics, n=3, tf=15, fps=120):
    frames = tf * fps
    t_eval = np.linspace(0, tf, frames)

    if n == 1:
        # Classic Single-Mass Pendulum
        t, l, m, g = sp.symbols('t l m g')
        q, p, a = sp.Function('q')(t), sp.Function('p')(t), sp.Function('a')(t)

        x =  l * sp.sin(q)
        y = -l * sp.cos(q)
        v_sqr = x.diff(t) ** 2 + y.diff(t) ** 2

        V = m * g * y
        T = m * v_sqr / 2
        L = T - V
        L = L.subs(q.diff(t), p)
        EL = EulerLagrange(L, q, p).subs(p.diff(t), a)
        simplifiedEL = EL.simplify().subs(q.diff(t), p).subs(p.diff(t), a)
        n1system = sp.solve(simplifiedEL, a)[0]
        ode1 = sp.utilities.lambdify([t, [q, p], l, m, g], [p, n1system])
        
        ode = partial(ode1, **params)
        sol = solve_ivp(ode, [0, tf], ics, t_eval=t_eval)

        ## Translating coordinates for convenience
        _q, _p = sol.y
        _l = params['l']
        x =   _l * np.sin(_q)
        y = - _l * np.cos(_q)
        return sol, [x, y]

    elif n == 2:
        # Classic Double-Mass Pendula
        t, l1, l2, m1, m2, g = sp.symbols('t l1 l2 m1 m2 g')
        q1, p1, a1 = sp.Function('q_1')(t), sp.Function('p_1')(t), sp.Function('a_1')(t)
        q2, p2, a2 = sp.Function('q_2')(t), sp.Function('p_2')(t), sp.Function('a_2')(t)

        x1 =  l1 * sp.sin(q1)
        y1 = -l1 * sp.cos(q1)
        x2 = x1 + l2 * sp.sin(q2)
        y2 = y1 - l2 * sp.cos(q2)
        v1_sqr = x1.diff(t) ** 2 + y1.diff(t) ** 2
        v2_sqr = x2.diff(t) ** 2 + y2.diff(t) ** 2

        V = m1 * g * y1 + m2 * g * y2
        T = m1 * v1_sqr / 2 + m2 * v2_sqr / 2
        L = T - V
        L = L.subs(q1.diff(t), p1).subs(q2.diff(t), p2).subs(p1.diff(t), a1).subs(p2.diff(t), a2)
        EL = EulerLagrange(L, [q1, q2], [p1, p2])

        simplifiedEL = [
            EL[0].simplify().subs(q1.diff(t), p1).subs(q2.diff(t), p2).subs(p1.diff(t), a1).subs(p2.diff(t), a2), 
            EL[1].simplify().subs(q1.diff(t), p1).subs(q2.diff(t), p2).subs(p1.diff(t), a1).subs(p2.diff(t), a2)
        ]
        n2system = sp.solve(simplifiedEL, a1, a2)
        ode2 = sp.utilities.lambdify([t, [q1, p1, q2, p2], l1, l2, m1, m2, g], [p1, n2system[a1], p2, n2system[a2]])
        ode = partial(ode2, **params)
        sol = solve_ivp(ode, [0, tf], ics, t_eval=t_eval)
        
        ## Translating coordinates for convenience
        _q1, _p1, _q2, _p2 = sol.y
        _l1, _l2 = params['l1'], params['l2']
        x1 =       _l1 * np.sin(_q1)
        y1 =     - _l1 * np.cos(_q1)
        x2 =  x1 + _l2 * np.sin(_q2)
        y2 =  y1 - _l2 * np.cos(_q2)        
        return sol, [x1, y1, x2, y2]

    elif n == 3:
        # Classic Triple-Mass Pendula
        t, l1, l2, l3, m1, m2, m3, g = sp.symbols('t l1 l2 l3 m1 m2 m3 g')
        q1, p1, a1 = sp.Function('q_1')(t), sp.Function('p_1')(t), sp.Function('a_1')(t)
        q2, p2, a2 = sp.Function('q_2')(t), sp.Function('p_2')(t), sp.Function('a_2')(t) 
        q3, p3, a3 = sp.Function('q_3')(t), sp.Function('p_3')(t), sp.Function('a_3')(t)

        x1 =      l1 * sp.sin(q1)
        y1 =    - l1 * sp.cos(q1)
        x2 = x1 + l2 * sp.sin(q2)
        y2 = y1 - l2 * sp.cos(q2)
        x3 = x2 + l3 * sp.sin(q3)
        y3 = y2 - l3 * sp.cos(q3)

        v1_sqr = x1.diff(t) ** 2 + y1.diff(t) ** 2
        v2_sqr = x2.diff(t) ** 2 + y2.diff(t) ** 2
        v3_sqr = x3.diff(t) ** 2 + y3.diff(t) ** 2

        V = m1 * g * y1 + m2 * g * y2 + m3 * g * y3
        T = m1 * v1_sqr / 2 + m2 * v2_sqr / 2 + m3 * v3_sqr / 2
        L = T - V
        L = L.subs(q1.diff(t), p1).subs(q2.diff(t), p2).subs(q3.diff(t), p3).subs(p1.diff(t), a1).subs(p2.diff(t), a2).subs(p3.diff(t), a3)
        EL = EulerLagrange(L, [q1, q2, q3], [p1, p2, p3])

        simplifiedEL = [
            EL[0].simplify().subs(q1.diff(t), p1).subs(q2.diff(t), p2).subs(q3.diff(t), p3).subs(p1.diff(t), a1).subs(p2.diff(t), a2).subs(p3.diff(t), a3), 
            EL[1].simplify().subs(q1.diff(t), p1).subs(q2.diff(t), p2).subs(q3.diff(t), p3).subs(p1.diff(t), a1).subs(p2.diff(t), a2).subs(p3.diff(t), a3),
            EL[2].simplify().subs(q1.diff(t), p1).subs(q2.diff(t), p2).subs(q3.diff(t), p3).subs(p1.diff(t), a1).subs(p2.diff(t), a2).subs(p3.diff(t), a3)
        ]
        n3system = sp.solve(simplifiedEL, a1, a2, a3)
        ode3 = sp.utilities.lambdify([t, [q1, p1, q2, p2, q3, p3], l1, l2, l3, m1, m2, m3, g], [p1, n3system[a1], p2, n3system[a2], p3, n3system[a3]])
        ode = partial(ode3, **params)
        sol = solve_ivp(ode, [0, tf], ics, t_eval=t_eval)
        
        ## Translating coordinates for convenience
        _q1, _p1, _q2, _p2, _q3, _p3 = sol.y
        _l1, _l2, _l3 = params['l1'], params['l2'], params['l3']
        x1 =      _l1 * sp.sin(_q1)
        y1 =    - _l1 * sp.cos(_q1)
        x2 = x1 + _l2 * sp.sin(_q2)
        y2 = y1 - _l2 * sp.cos(_q2)
        x3 = x2 + _l3 * sp.sin(_q3)
        y3 = y2 - _l3 * sp.cos(_q3)
        return sol, [x1, y1, x2, y2, x3, y3]

    else:
        raise NotImplementedError

In [ ]:
def pypendula_plot(sim_params, ani_params, rand_params, tf=15, n=3):
    if n == 1:
        # AA, BB = 1.6, 2.5  # min. = 1.53
        # N = 4

        # T_f, fps = 30, 60
        # frames = T_f * fps 
        # dt = T_f / frames

        # m, l, g = 1, 1, 9.81
        # t = np.linspace(0, T_f, frames)
        # xx = np.linspace(-np.pi, np.pi, 25)
        # yy = np.linspace(-2 * np.pi, 2 * np.pi, 25)
        # XX , YY = np.meshgrid(xx,yy)

        # def ode1(t, y):
        #     q, p = y
        #     return np.asarray([p, -g * np.sin(q)/l])

        # fig, (ax1, ax2) = plt.subplots(1, 2, squeeze=True, figsize=(20,10))
        # fig.suptitle('PyPendula')
        # ax1.set_aspect('equal')
        # ax1.set_xlim((-1.15 * l, 1.15 * l))
        # ax1.set_ylim((-1.15 * l, 1.15 * l))
        # ax1.set_xlabel('X [m]')
        # ax1.set_ylabel('Y [m]')
        # ax2.set_xlabel(r'$\theta$ [rad]')
        # ax2.set_ylabel(r'$\frac{d\theta}{dt}$ [rad]/[s]')
        # ax2.yaxis.tick_right()

        # DXX, DYY = ode1(t, [XX, YY])
        # M = (np.hypot(DXX, DYY))
        # M[M == 0] = 1.
        # DXX /= M
        # DYY /= M
        # ax2.quiver(XX, YY, DXX, DYY, M, pivot='mid')  # , cmap=plt.cm.plasma)

        # rand = np.asarray((BB - AA) * rng.random(size=(N, 2)) + AA)
        # ics = np.pi * np.power(rand, -np.ones_like(rand))
        # sols = [solve_ivp(ode1, [0, T_f], ic, t_eval=t) for ic in ics]
        # sol_plots = [ax2.plot(sol.y[0], sol.y[1], lw=1) for sol in sols]

        # xs = [ l * np.sin(sol.y[0]) for sol in sols]
        # ys = [-l * np.cos(sol.y[0]) for sol in sols]

        # line1, = ax1.plot([], [], 'o-', lw=2, color='red')
        # line2, = ax1.plot([], [], 'o-', lw=2, color='blue')
        # line3, = ax1.plot([], [], 'o-', lw=2, color='green')
        # line4, = ax1.plot([], [], 'o-', lw=2, color='gray')

        # def animate(i):
        #     this = [
        #         [[0, xs[0][i]], [0, ys[0][i]]],
        #         [[0, xs[1][i]], [0, ys[1][i]]],
        #         [[0, xs[2][i]], [0, ys[2][i]]],
        #         [[0, xs[3][i]], [0, ys[3][i]]],
        #     ]
        #     line1.set_data(this[0][0], this[0][1])
        #     line2.set_data(this[1][0], this[1][1])
        #     line3.set_data(this[2][0], this[2][1])
        #     line4.set_data(this[3][0], this[3][1])    
        #     return line1, line2, line3, line4,

        # anim = animation.FuncAnimation(fig, animate, len(t), interval=dt * 1000)
        # anim.save(f'./pypendula.mp4')
        pass
    
    if n == 2:
        frames = tf * ani_params['fps']
        dt = tf / frames
        t = np.linspace(0, tf, frames)
        rand_uniform = lambda a, b : (b - a) * rng.random() + a
    
        ## Solving
        ics = np.asarray([
            rand_uniform(rand_params['min_q'], rand_params['max_q']), 
            rand_uniform(rand_params['min_p'], rand_params['max_p']), 
            rand_uniform(rand_params['min_q'], rand_params['max_q']), 
            rand_uniform(rand_params['min_p'], rand_params['max_p'])
        ])
        sol, [x1, y1, x2, y2] = pypendula(params, ics, n=2)
        q1, p1, q2, p2 = sol.y

        ## Plotting
        fig, (ax1, ax2) = plt.subplots(1, 2, squeeze=True, figsize=(20,10))
        fig.suptitle('PyPendula')
        ax1.set_aspect('equal')
        ax1.set_xlim((-1.15 * (sim_params['l1'] + sim_params['l2']), 1.15 * (sim_params['l1'] + sim_params['l2'])))
        ax1.set_ylim((-1.15 * (sim_params['l1'] + sim_params['l2']), 1.15 * (sim_params['l1'] + sim_params['l2'])))
        ax1.set_xlabel('X [m]')
        ax1.set_ylabel('Y [m]')
        ax2.set_xlabel(r'$q$ [rad]')
        ax2.set_ylabel(r'$p$ [rad]/[s]')
        ax2.yaxis.tick_right()

        ax2.plot(q1, p1, lw=1, color='red')
        ax2.plot(q2, p2, lw=1, color='blue')

        point1, = ax2.plot([], [], 'o', lw=3, color='red')
        line1, = ax1.plot([], [], 'o-', lw=2, color='red', label=rf'$q_1(0)={ics[0]:.4f}, p_1(0)={ics[1]:.4f}$')
        point2, = ax2.plot([], [], 'o', lw=3, color='blue')
        line2, = ax1.plot([], [], 'o-', lw=2, color='blue', label=rf'$q_2(0)={ics[2]:.4f}, p_2(0)={ics[3]:.4f}$')
        ax1.legend()


        def animate(i):
            point2.set_data([q2[i]], [p2[i]])
            line2.set_data([x1[i], x2[i]], [y1[i], y2[i]])
            point1.set_data([q1[i]], [p1[i]])
            line1.set_data([0, x1[i]], [0, y1[i]])
            return point1, line1, point2, line2,


        anim = animation.FuncAnimation(fig, animate, len(t), interval=dt * 1000)
        anim.save(f'L:/lectures/Experiments/PyPendula/pypendula-n2-ics=[{ics[0]:.4f}, {ics[1]:.4f}, {ics[2]:.4f}, {ics[3]:.4f}].mp4')
        plt.close()
    elif n == 3:
        frames = tf * ani_params['fps']
        dt = tf / frames
        t = np.linspace(0, tf, frames)
        rand_uniform = lambda a, b : (b - a) * rng.random() + a
    
        ## Solving
        ics = np.asarray([
            rand_uniform(rand_params['min_q'], rand_params['max_q']), 
            rand_uniform(rand_params['min_p'], rand_params['max_p']), 
            rand_uniform(rand_params['min_q'], rand_params['max_q']), 
            rand_uniform(rand_params['min_p'], rand_params['max_p']),
            rand_uniform(rand_params['min_q'], rand_params['max_q']), 
            rand_uniform(rand_params['min_p'], rand_params['max_p']), 
        ])
        sol, [x1, y1, x2, y2, x3, y3] = pypendula(params, ics, n=3)
        q1, p1, q2, p2, q3, p3 = sol.y
    
        ## Plotting
        fig, (ax1, ax2) = plt.subplots(1, 2, squeeze=True, figsize=(20,10))
        fig.suptitle('PyPendula')
        ax1.set_aspect('equal')
        ax1.set_xlim((-1.15 * (params['l1'] + params['l2'] + params['l3']), 1.15 * (params['l1'] + params['l2'] + params['l3'])))
        ax1.set_ylim((-1.15 * (params['l1'] + params['l2'] + params['l3']), 1.15 * (params['l1'] + params['l2'] + params['l3'])))
        ax1.set_xlabel('X [m]')
        ax1.set_ylabel('Y [m]')
        ax2.set_xlabel(r'$q$ [rad]')
        ax2.set_ylabel(r'$p$ [rad]/[s]')
        ax2.yaxis.tick_right()

        ax2.plot(q1, p1, lw=1, color='red')
        ax2.plot(q2, p2, lw=1, color='blue')
        ax2.plot(q3, p3, lw=1, color='green')

        point1, = ax2.plot([], [], 'o', lw=3, color='red')
        line1, = ax1.plot([], [], 'o-', lw=2, color='red', label=rf'$q_1(0)={ics[0]:.6f}, p_1(0)={ics[1]:.6f}$')
        point2, = ax2.plot([], [], 'o', lw=3, color='blue')
        line2, = ax1.plot([], [], 'o-', lw=2, color='blue', label=rf'$q_2(0)={ics[2]:.6f}, p_2(0)={ics[3]:.6f}$')
        point3, = ax2.plot([], [], 'o', lw=3, color='green')
        line3, = ax1.plot([], [], 'o-', lw=2, color='green', label=rf'$q_3(0)={ics[4]:.6f}, p_3(0)={ics[5]:.6f}$')
        ax1.legend()


        def animate(i):
            point3.set_data([q3[i]], [p3[i]])
            line3.set_data([x2[i], x3[i]], [y2[i], y3[i]])
            point2.set_data([q2[i]], [p2[i]])
            line2.set_data([x1[i], x2[i]], [y1[i], y2[i]])
            point1.set_data([q1[i]], [p1[i]])
            line1.set_data([0, x1[i]], [0, y1[i]])
            return point1, line1, point2, line2, point3, line3,


        anim = animation.FuncAnimation(fig, animate, len(t), interval=dt * 1000)
        anim.save(f'L:/lectures/Experiments/PyPendula/pypendula-n3-ics=[{ics[0]:.6f}, {ics[1]:.6f}, {ics[2]:.6f}, {ics[3]:.6f}, {ics[4]:.6f}, {ics[5]:.6f}].mp4')
        plt.close()
        
        
    else:
        raise NotImplementedError